Import Panda und SQLalchemy

In [1]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
import psycopg2
from automasation_sh import add_state_year
from automasation_sh import to_str_fill_split

csv-File importieren.

In [2]:
sh_2022 = pd.read_csv("data/SH_2022.csv", delimiter= ";", encoding= 'windows-1252')
sh_2022.head(20)
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   EP               6004 non-null   int64  
 1   Kapitel          6004 non-null   int64  
 2   MGTG             2968 non-null   float64
 3   Titel            6004 non-null   int64  
 4   Zweckbestimmung  6004 non-null   object 
 5   Ist 2020 T€      6004 non-null   object 
 6   Soll 2021 T€     6004 non-null   object 
 7   Ansatz 2022 T€   6004 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 375.4+ KB


Drop die ersten zwei Columns, die nicht gebraucht werden

In [3]:
sh_2022 = sh_2022.drop(['Ist 2020 T€','Soll 2021 T€'], axis=1)

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [4]:
sh_2022.columns = sh_2022.columns.str.replace(" ", "_")
sh_2022.columns = sh_2022.columns.str.lower()
sh_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
sh_2022


,ep,kapitel,mgtg,titel,zweckbestimmung,ansatz_2022_euro
0,1,101,NaN,11902,Kostenbeiträge für private Benutzung verwaltun...,"0,0"
1,1,101,NaN,11904,Erlöse aus dem Verkauf unbrauchbarer oder entb...,"0,0"
2,1,101,NaN,11999,Vermischte Einnahmen,"0,0"
3,1,101,NaN,12401,"Einnahmen aus Vermietung, Verpachtung und Nutzung","0,0"
4,1,101,NaN,12402,Einnahmen aus der Inanspruchnahme von Leistung...,"18,1"
...,...,...,...,...,...,...
5999,16,1614,2.0,81222,Erwerb von Hard- und Software,"1.500,0"
6000,16,1614,5.0,81225,Erwerb von Hard- und Software,"0,0"
6001,16,1614,6.0,81226,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...","0,0"
6002,16,1614,7.0,81227,Erwerb digitaler Technologien,"1.200,0"


Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [5]:
sh_2022['ep'] = sh_2022['ep'].astype(str)
sh_2022['kapitel'] = sh_2022['kapitel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
sh_2022['ep'] = sh_2022['ep'].apply(lambda x:x.zfill(2))
sh_2022['kapitel'] = sh_2022['kapitel'].str[-2:]
sh_2022['kapitel'] = sh_2022['kapitel'].apply(lambda x:x.zfill(3))
sh_2022

,ep,kapitel,mgtg,titel,zweckbestimmung,ansatz_2022_euro
0,01,001,NaN,11902,Kostenbeiträge für private Benutzung verwaltun...,"0,0"
1,01,001,NaN,11904,Erlöse aus dem Verkauf unbrauchbarer oder entb...,"0,0"
2,01,001,NaN,11999,Vermischte Einnahmen,"0,0"
3,01,001,NaN,12401,"Einnahmen aus Vermietung, Verpachtung und Nutzung","0,0"
4,01,001,NaN,12402,Einnahmen aus der Inanspruchnahme von Leistung...,"18,1"
...,...,...,...,...,...,...
5999,16,014,2.0,81222,Erwerb von Hard- und Software,"1.500,0"
6000,16,014,5.0,81225,Erwerb von Hard- und Software,"0,0"
6001,16,014,6.0,81226,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...","0,0"
6002,16,014,7.0,81227,Erwerb digitaler Technologien,"1.200,0"


Bei den Zahlen . und , Systematik angepasst und *1000 genommen und von string zu float und dann INT umgewandelt.

In [6]:
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].str.replace('.', '').str.replace(",", ".").astype(float)
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].apply(lambda x:int(x*1000))

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1902/798112430.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].str.replace('.', '').str.replace(",", ".").astype(float)


Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [7]:
sh_2022["gruppe"] = sh_2022["titel"].str[:3]
sh_2022["counter"] = sh_2022["titel"].str[3:]
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ep                6004 non-null   object 
 1   kapitel           6004 non-null   object 
 2   mgtg              2968 non-null   float64
 3   titel             6004 non-null   object 
 4   zweckbestimmung   6004 non-null   object 
 5   ansatz_2022_euro  6004 non-null   int64  
 6   gruppe            6004 non-null   object 
 7   counter           6004 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 375.4+ KB


jahr und state_id eingefügt

In [8]:
sh_2022.insert(8, "jahr", 2022)
sh_2022.insert(9, "state_id", 15)
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ep                6004 non-null   object 
 1   kapitel           6004 non-null   object 
 2   mgtg              2968 non-null   float64
 3   titel             6004 non-null   object 
 4   zweckbestimmung   6004 non-null   object 
 5   ansatz_2022_euro  6004 non-null   int64  
 6   gruppe            6004 non-null   object 
 7   counter           6004 non-null   object 
 8   jahr              6004 non-null   int64  
 9   state_id          6004 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 469.2+ KB


titel gedropt, da redundant

In [9]:
sh_2022 = sh_2022.drop("titel", axis=1)
sh_2022.head(20)

,ep,kapitel,mgtg,zweckbestimmung,ansatz_2022_euro,gruppe,counter,jahr,state_id
0,01,001,NaN,Kostenbeiträge für private Benutzung verwaltun...,0,119,02,2022,15
1,01,001,NaN,Erlöse aus dem Verkauf unbrauchbarer oder entb...,0,119,04,2022,15
2,01,001,NaN,Vermischte Einnahmen,0,119,99,2022,15
3,01,001,NaN,"Einnahmen aus Vermietung, Verpachtung und Nutzung",0,124,01,2022,15
4,01,001,NaN,Einnahmen aus der Inanspruchnahme von Leistung...,18100,124,02,2022,15
5,01,001,NaN,Einnahmen aus Veranstaltungen nach dem Nutzung...,8500,124,03,2022,15
6,01,001,NaN,Einnahmen aus der Beteiligung Dritter bei Kuns...,0,129,01,2022,15
7,01,001,1.0,Erstattung von Personalkosten,0,281,02,2022,15
8,01,001,NaN,Entnahme aus der Rücklage für Sachausgaben,0,359,01,2022,15
9,01,001,NaN,Entnahme aus der Rücklage für personalwirtscha...,0,359,02,2022,15


anhand der gruppe einnahmen und ausgaben in den Zahlen definiert

In [10]:
sh_2022['ansatz_2022_euro'] =sh_2022[['ansatz_2022_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)


mgtg gedropt, hat keinen mehrwert und viele nan

In [11]:
sh_2022 = sh_2022.drop(['mgtg'], axis=1)


einmal angucken, da schön

In [12]:
sh_2022.head()

,ep,kapitel,zweckbestimmung,ansatz_2022_euro,gruppe,counter,jahr,state_id
0,01,001,Kostenbeiträge für private Benutzung verwaltun...,0,119,02,2022,15
1,01,001,Erlöse aus dem Verkauf unbrauchbarer oder entb...,0,119,04,2022,15
2,01,001,Vermischte Einnahmen,0,119,99,2022,15
3,01,001,"Einnahmen aus Vermietung, Verpachtung und Nutzung",0,124,01,2022,15
4,01,001,Einnahmen aus der Inanspruchnahme von Leistung...,18100,124,02,2022,15


Wir bereiten alles für den Push in die Database vor.

In [13]:
get_sql_config()
engine = get_engine()



Nun können wir pushen.

In [14]:
#push_to_database(sh_2022, "budget_text_sh", engine , "capstone_public_budgeting")

Wir ergänzen das nun hinsichtlich der Codierungstabellen von SH: Einlesen und hochladen.

In [15]:
codierung_sh_2022_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2022.csv")
codierung_sh_2021_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2021.csv")
codierung_sh_2020_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2020.csv")
codierung_sh_2019_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2019.csv")
codierung_sh_2018_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2018.csv")
codierung_sh_2022_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2022.csv")
codierung_sh_2021_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2021.csv")
codierung_sh_2020_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2020.csv")
codierung_sh_2019_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2019.csv")
codierung_sh_2018_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2018.csv")

Nun müssen die jeweiligen df aufbereitet werden. Wir fangen mit den EP + Bezeichnungen an. Int muss zu str + leading 0 + Columnname anpassen.

In [16]:
codierung_sh_2022_EP['ep_nr'] = codierung_sh_2022_EP['ep_nr'].astype(str)
codierung_sh_2022_EP['ep_nr'] = codierung_sh_2022_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2022_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2022_EP.insert(len(codierung_sh_2022_EP.columns), "state_id", 15)

codierung_sh_2021_EP['ep_nr'] = codierung_sh_2021_EP['ep_nr'].astype(str)
codierung_sh_2021_EP['ep_nr'] = codierung_sh_2021_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2021_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2021_EP.insert(len(codierung_sh_2021_EP.columns), "state_id", 15)

codierung_sh_2020_EP['ep_nr'] = codierung_sh_2020_EP['ep_nr'].astype(str)
codierung_sh_2020_EP['ep_nr'] = codierung_sh_2020_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2020_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2020_EP.insert(len(codierung_sh_2020_EP.columns), "state_id", 15)

codierung_sh_2019_EP['ep_nr'] = codierung_sh_2019_EP['ep_nr'].astype(str)
codierung_sh_2019_EP['ep_nr'] = codierung_sh_2019_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2019_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2019_EP.insert(len(codierung_sh_2019_EP.columns), "state_id", 15)

codierung_sh_2018_EP['ep_nr'] = codierung_sh_2018_EP['ep_nr'].astype(str)
codierung_sh_2018_EP['ep_nr'] = codierung_sh_2018_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2018_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2018_EP.insert(len(codierung_sh_2018_EP.columns), "state_id", 15)


In [17]:
codierung_sh_2022_EP

,ep,ep_bezeichnung,jahr,state_id
0,01,Landtag,2022,15
1,02,Landesrechnungshof,2022,15
2,03,Ministerpräsident; Staatskanzlei,2022,15
3,04,Ministerium für Inneres; ländliche Räume; Inte...,2022,15
4,05,Finanzministerium,2022,15
5,06,Ministerium für Wirtschaft; Verkehr; Arbeit; T...,2022,15
6,07,Ministerium für Bildung; Wissenschaft und Kultur,2022,15
7,09,Ministerium für Justiz; Europa und Verbraucher...,2022,15
8,10,Ministerium für Soziales; Gesundheit; Jugend; ...,2022,15
9,11,Allgemeine Finanzverwaltung,2022,15


Nun kümmern wir uns um die Kapitel. Wieder int zu str, leading 0 und namen anpassen.

In [18]:
codierung_sh_2022_KNr['ep_nr'] = codierung_sh_2022_KNr['ep_nr'].astype(str)
codierung_sh_2022_KNr['kapitel_nr'] = codierung_sh_2022_KNr['kapitel_nr'].astype(str)
codierung_sh_2022_KNr['ep_nr'] = codierung_sh_2022_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2022_KNr['kapitel_nr'] = codierung_sh_2022_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2022_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2022_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2022_KNr.insert(len(codierung_sh_2022_KNr.columns), "state_id", 15)

codierung_sh_2021_KNr['ep_nr'] = codierung_sh_2021_KNr['ep_nr'].astype(str)
codierung_sh_2021_KNr['kapitel_nr'] = codierung_sh_2021_KNr['kapitel_nr'].astype(str)
codierung_sh_2021_KNr['ep_nr'] = codierung_sh_2021_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2021_KNr['kapitel_nr'] = codierung_sh_2021_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2021_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2021_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2021_KNr.insert(len(codierung_sh_2021_KNr.columns), "state_id", 15)

codierung_sh_2020_KNr['ep_nr'] = codierung_sh_2020_KNr['ep_nr'].astype(str)
codierung_sh_2020_KNr['kapitel_nr'] = codierung_sh_2020_KNr['kapitel_nr'].astype(str)
codierung_sh_2020_KNr['ep_nr'] = codierung_sh_2020_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2020_KNr['kapitel_nr'] = codierung_sh_2020_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2020_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2020_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2020_KNr.insert(len(codierung_sh_2020_KNr.columns), "state_id", 15)

codierung_sh_2019_KNr['ep_nr'] = codierung_sh_2019_KNr['ep_nr'].astype(str)
codierung_sh_2019_KNr['kapitel_nr'] = codierung_sh_2019_KNr['kapitel_nr'].astype(str)
codierung_sh_2019_KNr['ep_nr'] = codierung_sh_2019_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2019_KNr['kapitel_nr'] = codierung_sh_2019_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2019_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2019_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2019_KNr.insert(len(codierung_sh_2019_KNr.columns), "state_id", 15)

codierung_sh_2018_KNr['ep_nr'] = codierung_sh_2018_KNr['ep_nr'].astype(str)
codierung_sh_2018_KNr['kapitel_nr'] = codierung_sh_2018_KNr['kapitel_nr'].astype(str)
codierung_sh_2018_KNr['ep_nr'] = codierung_sh_2018_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2018_KNr['kapitel_nr'] = codierung_sh_2018_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2018_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2018_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2018_KNr.insert(len(codierung_sh_2018_KNr.columns), "state_id", 15)

In [19]:
codierung_sh_2018_KNr

,ep,kapitel,kapitel_bezeichnung,jahr,state_id
0,01,001,Landtag,2018,15
1,01,002,Unabhängiges Landeszentrum für Datenschutz,2018,15
2,01,003,Die Bürgerbeauftragte für soziale Angelegenhei...,2018,15
3,01,004,Der Beauftragte für Flüchtlings-; Asyl- und Zu...,2018,15
4,01,005,Die Landesbeauftragte für Menschen mit Behinde...,2018,15
...,...,...,...,...,...
118,16,010,Ministerium für Soziales; Gesundheit; Jugend; ...,2018,15
119,16,011,Allgemeine Finanzverwaltung,2018,15
120,16,012,Hochbaumaßnahmen und Raumbedarfsdeckung des La...,2018,15
121,16,013,Ministerium für Energiewende; Landwirtschaft; ...,2018,15


Es klappt! Nachfolgend immer das jeweilige df einfügen. ausge#, da sonst doppelt und dreifach hochgeladen.

In [20]:
#push_to_database(codierung_sh_2018_EP,"codierung_sh_EP", engine, "capstone_public_budgeting")
#push_to_database(codierung_sh_2018_KNr,"codierung_sh_KNr", engine, "capstone_public_budgeting")

In [21]:
sh_2022.rename(columns={'jahr':'year', 'ansatz_2022_euro':'amount'}, inplace=True)
sh_2022

,ep,kapitel,zweckbestimmung,amount,gruppe,counter,year,state_id
0,01,001,Kostenbeiträge für private Benutzung verwaltun...,0,119,02,2022,15
1,01,001,Erlöse aus dem Verkauf unbrauchbarer oder entb...,0,119,04,2022,15
2,01,001,Vermischte Einnahmen,0,119,99,2022,15
3,01,001,"Einnahmen aus Vermietung, Verpachtung und Nutzung",0,124,01,2022,15
4,01,001,Einnahmen aus der Inanspruchnahme von Leistung...,18100,124,02,2022,15
...,...,...,...,...,...,...,...,...
5999,16,014,Erwerb von Hard- und Software,-1500000,812,22,2022,15
6000,16,014,Erwerb von Hard- und Software,0,812,25,2022,15
6001,16,014,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",0,812,26,2022,15
6002,16,014,Erwerb digitaler Technologien,-1200000,812,27,2022,15


In [22]:
#push_to_database(sh_2022, "budget_all", engine, "capstone_public_budgeting")

The budget_all table was imported successfully.


In [26]:
codierung_sh_2022_KNr.rename(columns={'jahr':'year', 'kapitel_bezeichnung':'kapitel_txt'}, inplace=True)
codierung_sh_2022_KNr

,ep,kapitel,kapitel_txt,year,state_id
0,01,001,Landtag,2022,15
1,01,002,Unabhängiges Landeszentrum für Datenschutz,2022,15
2,01,003,Die Bürgerbeauftragte für soziale Angelegenhei...,2022,15
3,01,004,Der Beauftragte für Flüchtlings-; Asyl- und Zu...,2022,15
4,01,005,Die Landesbeauftragte für Menschen mit Behinde...,2022,15
...,...,...,...,...,...
122,16,010,Ministerium für Soziales; Gesundheit; Jugend; ...,2022,15
123,16,011,Allgemeine Finanzverwaltung,2022,15
124,16,012,Hochbaumaßnahmen und Raumbedarfsdeckung des La...,2022,15
125,16,013,Ministerium für Energiewende; Landwirtschaft; ...,2022,15


In [27]:
#push_to_database(codierung_sh_2022_KNr, "kapitel", engine, "capstone_public_budgeting")

The kapitel table was imported successfully.


In [29]:
codierung_sh_2022_EP.rename(columns={'jahr':'year', 'ep_bezeichnung':'ep_txt'}, inplace=True)
codierung_sh_2022_EP

,ep,ep_txt,year,state_id
0,01,Landtag,2022,15
1,02,Landesrechnungshof,2022,15
2,03,Ministerpräsident; Staatskanzlei,2022,15
3,04,Ministerium für Inneres; ländliche Räume; Inte...,2022,15
4,05,Finanzministerium,2022,15
5,06,Ministerium für Wirtschaft; Verkehr; Arbeit; T...,2022,15
6,07,Ministerium für Bildung; Wissenschaft und Kultur,2022,15
7,09,Ministerium für Justiz; Europa und Verbraucher...,2022,15
8,10,Ministerium für Soziales; Gesundheit; Jugend; ...,2022,15
9,11,Allgemeine Finanzverwaltung,2022,15


In [30]:
#push_to_database(codierung_sh_2022_EP, "einzelplaene", engine, "capstone_public_budgeting")

The einzelplaene table was imported successfully.
